In [44]:
#Thomas Conibear - Second Homework on LJ Potential

#Code straight from notes used to find LJ potential for multiple atom systems.

import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
import requests

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

def dist(N_atoms):
    pos = get_pos_from_url(N = N_atoms)
    pos = np.reshape(pos,[N_atoms,3])
    #pos = 10*np.random.random(N_atoms*3)
    #pos = np.reshape(pos,[N_atoms,3])
    distance = cdist(pos,pos,'euclidean')
    iu1 = np.triu_indices(N_atoms)
    distance = distance[iu1]
    index = np.argwhere(distance==0)
    distance = np.delete(distance,index)
    return distance
    
def LJ(r):
    r6 = np.power(r,6)
    r12 = np.multiply(r6,r6)
    return 4*(1/r12 - 1/r6)
    
def Total_Energy(positions):

    E = LJ(positions)
    Energy = np.sum(E)
    return Energy

N_attempts = 50
f_values = []
x_values = []

#First Scipy optimize method
def ground_state_potential(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='CG', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, " Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))

#Choosing new method of Nelder-Mead to compare results

def ground_state_potential2(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='Nelder-Mead', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))
    
#Choosing another method of Powell to compare results

def ground_state_potential3(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='Powell', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))
    
#Choosing the fourth method of BFGS to compare results
def ground_state_potential4(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='BFGS', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))

In [40]:
%%time

print("Results for Scipy Minimization Method Using CG:", "\n")
ground_state_potential(20)

Results for Scipy Minimization Method Using CG: 

Step:  0   Value: -189.99999999999903  Number of Iterations:  59
Step:  10   Value: -189.99999999999903  Number of Iterations:  59
Step:  20   Value: -189.99999999999903  Number of Iterations:  59
Step:  30   Value: -189.99999999999903  Number of Iterations:  59
Step:  40   Value: -189.99999999999903  Number of Iterations:  59
The ground state potential is:   -189.99999999999903
Wall time: 3min 57s


In [46]:
%%time
print("Results for Scipy Minimization Method Using Nelder-Mead:", "\n")
ground_state_potential2(15)

#Nelder-Mead method has a large amount of iterations so I only used a 15 atom system here

Results for Scipy Minimization Method Using Nelder-Mead: 

Step:  0   Value: -69.04514235542156 Number of Iterations:  19535
Step:  10   Value: -69.04514235542156 Number of Iterations:  19535
Step:  20   Value: -69.04514235542156 Number of Iterations:  19535
Step:  30   Value: -69.04514235542156 Number of Iterations:  19535
Step:  40   Value: -69.04514235542156 Number of Iterations:  19535
The ground state potential is:   -69.04514235542156
Wall time: 8min 27s


In [48]:
%%time
print("Results for Scipy Minimization Method Using Powell:", "\n")
ground_state_potential3(20)

#We can see that the Powell method is the fastest for this type of calculation

Results for Scipy Minimization Method Using Powell: 

Step:  0   Value: -189.99999999999721 Number of Iterations:  3
Step:  10   Value: -189.99999999999721 Number of Iterations:  3
Step:  20   Value: -189.99999999999721 Number of Iterations:  3
Step:  30   Value: -189.99999999999721 Number of Iterations:  3
Step:  40   Value: -189.99999999999721 Number of Iterations:  3
The ground state potential is:   -189.99999999999721
Wall time: 1min 44s


In [45]:
%%time
print("Results for Scipy Minimization Method Using BFGS:", "\n")
ground_state_potential4(3)

#Used a 3 atom system to compare to the previous homework. It is clear that the random method I used had high error

Results for Scipy Minimization Method Using BFGS: 

Step:  0   Value: -2.9999999999980176 Number of Iterations:  0
Step:  10   Value: -2.9999999999980176 Number of Iterations:  0
Step:  20   Value: -2.9999999999980176 Number of Iterations:  0
Step:  30   Value: -2.9999999999980176 Number of Iterations:  0
Step:  40   Value: -2.9999999999980176 Number of Iterations:  0
The ground state potential is:   -2.9999999999980176
Wall time: 326 ms
